In [91]:
import numpy as np, ephem
from numpy.lib import recfunctions
from pixell import utils, enmap, bunch, reproject, colors, coordinates#, mpi
from scipy import interpolate, optimize
import glob
import matplotlib.pyplot as plt
import matplotlib
import pickle as pk

matplotlib.use('pdf')

from pathlib import Path
from asteroid_utils_pixell import minorplanet, get_desig

from astroquery.mpc import MPC

import astropy.units as u

In [88]:
def get_decs(i):
    desig, name, semimajor = get_desig(i)
    try:
        info    = np.load('/gpfs/fs0/project/r/rbond/sigurdkn/actpol/ephemerides/objects/{}.npy'.format(name.capitalize())).view(np.recarray)
    except FileNotFoundError:
        return 0
    orbit   = interpolate.interp1d(info.ctime, 
                                   [utils.unwind(info.ra*utils.degree), info.dec*utils.degree,
                                    info.r, info.ang*utils.arcsec], kind=3)
    result = MPC.query_object('asteroid', name=name, return_fields="period")
    period = float(result[0]["period"])
    times = np.arange(info.ctime[0], info.ctime[0] + period * (365.25*24*60*60), 60*60*24)
    try:
        dec =  orbit(times)[1]
    except ValueError:
        dec = 0
    return dec

In [89]:
decs = np.array([])
for i in range(1150):
    print(i, end="\r")
    dec = get_decs(i)
    decs = np.append(decs, dec)


In [95]:
(np.percentile(decs, [10, 90])*u.radian).to(u.deg)

<Quantity [-24.37303716,  24.2446278 ] deg>

In [71]:
np.min(decs)

-4393214.219589233

In [2]:
from astropy import units as u

from astropy.cosmology import Planck15

import numpy as np

from pixell import utils

In [5]:
freq = 280 * u.GHz

equiv = u.thermodynamic_temperature(freq, Planck15.Tcmb0)

beam_fwhm = 0.9  *u.arcmin
fwhm_to_sigma = 1. / (8 * np.log(2))**0.5
beam_sigma = beam_fwhm * fwhm_to_sigma
omega_B = 2 * np.pi * beam_sigma**2

uk_beam = (37/np.sqrt(3) * u.uK *u.arcmin)/ beam_fwhm

Jy_sr = uk_beam.to(u.K).value*utils.planck(freq.to(u.Hz).value, utils.T_cmb)

mJy_sr = uk_beam.to(u.mJy / u.sr, equivalencies=equiv)
mJy_sr.to(u.mJy/u.beam, equivalencies=u.beam_angular_area(omega_B))*5

<Quantity 0.791037 mJy / beam>

In [6]:
(Jy_sr*u.Jy/u.sr).to(u.mJy/u.beam, equivalencies=u.beam_angular_area(omega_B))

<Quantity 0.43410414 mJy / beam>

In [10]:
5.8/np.sqrt(3)

3.3486315612998294

In [19]:
utils.planck(freq.to(u.Hz).value, utils.T_cmb)

235501495.32492262

In [20]:
freq = 23*u.GHz
utils.planck(freq.to(u.Hz).value, utils.T_cmb)

35930329.916144244

5589.748348505475

235501495.32492262

429137569.30085653